In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, models
from torchvision.transforms import ToTensor, Pad, Compose, ToPILImage, Normalize, ConvertImageDtype, Resize
from torch import nn
import torch.nn.functional as F
import pandas as pd 
import os
import time
import copy
import matplotlib.pyplot as plt
from torchvision.models import resnet18
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
#Dataset

from torchvision.io import read_image

class GTSRB(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[["Path","ClassId"]]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
#Dataset Initialization
img_dir = "/kaggle/input/gtsrb-german-traffic-sign/"
test_file = "/kaggle/input/gtsrb-german-traffic-sign/Test.csv"
test_data = GTSRB(img_dir = img_dir, annotations_file = test_file,
                   transform = Compose([Resize((30,30)), ConvertImageDtype(torch.float32)]))
#Prepare DataLoader
b_size = 1
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_data, batch_size=b_size, shuffle=False)

In [ ]:
# Display image and label.
test_features, test_labels = next(iter(test_dataloader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")
img = test_features[0]
label = test_labels[0]
img = ToPILImage()(img).convert("RGB")
plt.imshow(img)
plt.show()
print(f"Label: {label}")

In [ ]:
#Training Module
def test(model, test_dataloader, output=None):
    preds_list = []
    labels_list = []
    acc = 0.0
    since = time.time()
    running_corrects = 0
    
    dataset_size = len(test_dataloader)*b_size 
    pbar = tqdm(total=dataset_size)

    # Iterate over data.
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.no_grad():
            outputs = model(inputs)    
            _, preds = torch.max(outputs.data, 1)
            preds_list.append(preds)
            labels_list.append(labels)

        # statistics
        
        running_corrects += torch.sum(preds == labels)
        pbar.update(len(inputs))
     
    acc = running_corrects.double()/ dataset_size

    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    print('Acc: {:.4f}'.format(acc))
          
    return preds_list, labels_list

In [ ]:
weight_dir = '/kaggle/input/weight-resnet18/ResNet18_best.pth'
model = torch.load(weight_dir, map_location=torch.device('cpu'))
model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("You are using Device: ", device)
model.to(device)

In [ ]:
preds_list, labels_list = test(model, test_dataloader)

In [ ]:
preds_list_one = F.one_hot(torch.tensor(preds_list), num_classes=43)
labels_list_one = F.one_hot(torch.tensor(labels_list), num_classes=43)


In [ ]:
average_precision_score(preds_list_one.numpy(), labels_list_one.numpy())